<center><p float="center">
  <img src="https://upload.wikimedia.org/wikipedia/commons/e/e9/4_RGB_McCombs_School_Brand_Branded.png" width="300" height="100"/>
  <img src="https://mma.prnewswire.com/media/1458111/Great_Learning_Logo.jpg?p=facebook" width="200" height="100"/>
</p></center>

<center><font size=10>Generative AI for Business Applications</center></font>
<center><font size=6>Fine-Tunning LLMs - Week 1</center></font>

<center><p float="center">
  <img src="" width=720></a>
<center><font size=6>Fine-Tuned AI for Summarizing Insurance Sales Conversations</center></font>

# Problem Statement

## Business Context

An enterprise sales representative at a global insurance provider is preparing for a crucial renewal meeting with one of the largest clients. Over the past year, numerous emails have been exchanged, several calls conducted, and in-person meetings held. However, this valuable context is fragmented across the inbox, CRM records, and call notes.

With limited time and growing pressure to personalize service and identify cross-sell opportunities, it is difficult to recall key details, such as the products the client was interested in, concerns raised in the last quarter, and commitments made during previous meetings.

This challenge reflects a broader industry problem where client interactions are rich but scattered. Sales teams often face:

* **Overload of unstructured data** from emails, calls, and notes.
* **Lack of standardized, accurate summaries** to capture client context.
* **Manual, error-prone preparation** that consumes significant time.
* **Missed upsell and personalization opportunities**, weakening client trust.

As a result, client engagement is inconsistent, preparation is inefficient, and revenue opportunities are lost.



##  Objective

The objective is to introduce a **smart assistant** capable of synthesizing multi-modal client interactions and generating precise, context-aware summaries.

Such a solution would:

* Consolidate insights from emails, CRM logs, call transcripts, and meeting notes.
* Deliver concise, tailored client briefs before every touchpoint.
* Help sales teams maintain continuity, honor past commitments, and personalize conversations.
* Unlock new revenue by surfacing upsell and cross-sell opportunities at the right moment.

By reducing preparation time and improving personalization, this assistant can transform client engagement in the insurance sector, strengthen relationships, and drive sustainable growth.

## Data Description

The dataset consists of two primary columns:

Conversation - Contains the raw transcripts of client-sales representative interactions, which are often lengthy, multi-turn, and unstructured.

Summary - Provides the corresponding concise, structured summaries of key discussion points, client interests, concerns, and commitments.

# **Solution Approach**
Provide a Custom Fine-Tuned AI Model for Sales Interaction Summarization

To address this challenge, we propose training a domain-specific fine-tuned language model tailored for enterprise insurance communication.
The model will:

1. Ingest few multi-modal inputs (emails, transcripts, notes).
2. Identify intent, extract key discussion points, client interests, pain points, and commitments.
3. Generate concise, actionable summaries under 200 words, customized for enterprise insurance workflows.
4. Be fine-tuned on real-world communication data to learn domain-specific vocabulary and interaction patterns.

This AI-powered tool will augment sales productivity, enhance client engagement, and ensure consistent follow-ups, turning scattered conversations into strategic intelligence.

# **Installing and Importing Necessary Libraries**

In [ ]:
!pip install --no-deps bitsandbytes accelerate xformers==0.0.32.post2 peft trl==0.15.2 triton cut_cross_entropy unsloth_zoo
!pip install sentencepiece protobuf huggingface_hub hf_transfer
!pip install transformers==4.51.3
!pip install --no-deps unsloth

!pip install -q datasets evaluate bert-score

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.2/117.2 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 MB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.8/184.8 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 79.2 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.55.2
    Uninstalling transformers-4.55.2:
      Successfully uninstalled transformers-4.55.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
unsloth-zoo 2025.8.8 requires msgspec, which is not installed.
unsloth-zoo 2025.8.8 requires tyro, which is not installed.
unsloth-zoo 2025.8.8 requires datasets<4.0.0,>=3.4.1, but you have datasets 4.0.0 which is incompatible.
     ━━━━━━━━━━━━━━

**Note**:
- After running the above cell, kindly restart the runtime (for Google Colab) or notebook kernel (for Jupyter Notebook), and run all cells sequentially from the next cell.
- On executing the above line of code, you might see a warning regarding package dependencies. This error message can be ignored as the above code ensures that all necessary libraries and their dependencies are maintained to successfully execute the code in ***this notebook***.

In [ ]:
from unsloth import FastLanguageModel          # Efficient fine-tuning and inference utilities for LLMs (optimized training).
import torch                                   # Core deep learning library for tensor operations and model training.
import evaluate                                # Hugging Face library for evaluating NLP models with standard metrics.
from tqdm import tqdm                          # Progress bar utility for tracking loops and training progress.
import pandas as pd                            # Data manipulation and analysis library (tabular data handling).
from datasets import Dataset                   # Hugging Face library for creating and managing ML datasets.

from trl import SFTTrainer                     # Trainer class for supervised fine-tuning of language models.
from transformers import TrainingArguments, EarlyStoppingCallback, DataCollatorForSeq2Seq
# - TrainingArguments: configuration for training (batch size, learning rate, etc.)
# - EarlyStoppingCallback: stops training when validation performance stops improving.
# - DataCollatorForSeq2Seq: prepares batches of data for sequence-to-sequence models.

from unsloth import is_bfloat16_supported      # Utility to check if bfloat16 precision is supported on your hardware.


# **1. Evaluation of LLM before FineTuning**

### Loading the Testing Data


In [ ]:
# Read the testing CSV into a Pandas DataFrame
testing_data = pd.read_csv("/content/finetuning_testing.csv")

# Extract all dialogues into a list for model input
test_dialogues = [sample for sample in testing_data['Dialogues']]

# Extract all human-written summaries into a list for evaluation
test_summaries = [sample for sample in testing_data['Summary']]

### Loading the Mistral Model

In [ ]:
# Load the pre-trained Mistral model with 4-bit quantization for efficiency
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/mistral-7b-instruct-v0.2-bnb-4bit",  # Model checkpoint to load
    max_seq_length=4096,  # Maximum sequence length the model can handle
    dtype=None,            # Use default data type (automatic)
    load_in_4bit=True      # Load the model in 4-bit precision to save memory
)


==((====))==  Unsloth 2025.8.9: Fast Mistral patching. Transformers: 4.51.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.4.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.32.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [ ]:
# Prepare the model for inference (generating predictions)
FastLanguageModel.for_inference(model)

MistralForCausalLM(
  (model): MistralModel(
    (embed_tokens): Embedding(32000, 4096, padding_idx=0)
    (layers): ModuleList(
      (0-31): 32 x MistralDecoderLayer(
        (self_attn): MistralAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): MistralMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): MistralRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): Mis

### Inference


The Alpaca instruction prompt is a general purpose prompt template that can be adapted to any task.

In [ ]:
alpaca_prompt_template = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Write a concise summary of the following dialogue.

### Input:
{}

### Response:
{}
"""

In [ ]:
# Initialize an empty list to store the summaries generated by the model
predicted_summaries = []

We are generating summaries for each dialogue in our test set using the fine-tuned model.

**Step-by-step Approach:**

1. **Iterate through test dialogues** - `for dialogue in tqdm(test_dialogues):`

   * Loops through each test dialogue while showing a progress bar (`tqdm`).

2. **Format the prompt**

   * Inserts the dialogue into the summarization template.

3. **Tokenize input**

   * Converts the text prompt into tokens (numbers) and moves them to the GPU (`.to("cuda")`).

4. **Generate output**

   * The model predicts the summary using `.generate()`.
   * `max_new_tokens=128`: limits summary length.
   * `temperature=0`: makes output deterministic (no randomness).
   * `pad_token_id`: ensures proper padding using EOS token.

5. **Decode output**

   * Converts model tokens back into human-readable text.
   * Skips special tokens and cleans formatting.

6. **Store prediction**

   * Appends the generated summary to `predicted_summaries`.

7. **Error handling**

   * If an error occurs, it prints the error and continues with the next dialogue instead of stopping.

This loop **takes each dialogue -> feeds it to the model -> generates a summary -> saves it for evaluation**.

In [ ]:
# Loop through each dialogue in the test set and generate summaries
for dialogue in tqdm(test_dialogues):

    try:
        # Format the dialogue into the Alpaca-style prompt template
        prompt = alpaca_prompt_template.format(dialogue, " ")

        # Tokenize the prompt and move it to GPU
        inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

        # Generate summary with the model
        outputs = model.generate(
            **inputs,
            max_new_tokens=128,          # Limit summary length
            use_cache=True,              # Speed up generation
            temperature=0,               # Deterministic output
            pad_token_id=tokenizer.eos_token_id
        )

        # Decode the generated tokens into human-readable text
        prediction = tokenizer.decode(
            outputs[0][inputs.input_ids.shape[-1]:],  # Skip prompt tokens
            skip_special_tokens=True,
            cleanup_tokenization_spaces=True
        )

        # Store the generated summary
        predicted_summaries.append(prediction)

    except Exception as e:
        print(e)  # Log any errors and continue with next dialogue
        continue


100%|██████████| 10/10 [01:16<00:00,  7.62s/it]


### Evaluation


Now we are evaluating our base model to check how well the generated summaries align with human-written summaries. For this, we are using BERTScore, which measures the semantic similarity between the two.

**BERTScore** is a metric for evaluating text generation tasks, including summarization, translation, and captioning. Unlike traditional metrics like ROUGE or BLEU that rely on exact word overlaps, BERTScore uses embeddings from a pre-trained BERT model to measure **semantic similarity** between the generated text (predictions) and the human-written text (references). This makes it more robust in capturing meaning, even when different words are used.

* **Precision** - Measures how much of the content in the generated text is actually relevant to the reference. High precision means the model is not adding irrelevant or “extra” information.

* **Recall** - Measures how much of the important content from the reference is captured by the generated text. A high recall means the model covers most of the key points, even if it includes some extra details.

* **F1 Score** - Combines both precision and recall into a balanced score. It demonstrates how well the generated text both covers the important content and remains relevant. This is usually reported as the main metric for BERTScore.

In short, BERTScore helps evaluate not just word matching, but whether the **meaning** of the generated text aligns with the reference.


We are proceeding with the F1-Score, as it provides a balanced measure of the overall semantic similarity.

In [ ]:
# Load the BERTScore evaluation metric from the Hugging Face 'evaluate' library
bert_scorer = evaluate.load("bertscore")

Hyperparameters for `bert_scorer`

* **`predictions`** - The summaries generated by our fine-tuned model.
* **`references`** - The correct (gold-standard) summaries from the dataset.
* **`lang`='en'** - Specifies the language as English.
* **`rescale_with_baseline`=True** - Normalizes the scores so they are easier to interpret.



In [ ]:
# Compute BERTScore for the model's generated summaries
score = bert_scorer.compute(
    predictions=predicted_summaries,   # Summaries generated by the model
    references=test_summaries,         # Human-written reference summaries
    lang='en',                         # Language of the summaries
    rescale_with_baseline=True         # Normalize scores for easier interpretation
)

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Now we calculate the **average F1 score** across all evaluated summaries, giving an overall performance measure of the model.


**Note:** Since this is a generative model, the output may vary slightly each time. Additionally, because the evaluator is built on neural networks, its responses may also change.

In [ ]:
# Calculate the average F1 score across all generated summaries
average_f1 = sum(score['f1']) / len(score['f1'])
average_f1

0.21308062300086023

**The BERT Score of Mistral LLM is 0.21**


# **2. Fine Tuning LLM**

## Data Preparation

We first read the CSV into a **Pandas DataFrame** because it is easy to inspect and manipulate tabular data. However, Hugging Face models and trainers do not work directly with DataFrames they expect data in the form of a **`Dataset` object** from the `datasets` library.

That’s why we convert the DataFrame into a **dictionary of lists**. The `Dataset.from_dict()` method then turns this dictionary into a Hugging Face `Dataset`, which is optimized for:

* fast tokenization, shuffling, and batching,
* direct compatibility with `Trainer` / `SFTTrainer`,
* efficient storage and processing on large datasets.

DataFrame stores data like a table (rows × columns), while a Dataset stores data as a dictionary of columns (each column is an array/list), making it better suited for ML pipelines.

#### Load the Dataset

In [ ]:
# Read the fine-tuning training CSV into a Pandas DataFrame
training = pd.read_csv("/content/finetuning_training.csv")

# Convert the DataFrame into a dictionary of lists (required for Hugging Face Dataset)
training_dict = training.to_dict(orient='list')

# Create a Hugging Face Dataset from the dictionary
training_dataset = Dataset.from_dict(training_dict)

Store the end-of-sequence token (used to mark the end of each input/output text)

In [ ]:
# Get the end-of-sequence (EOS) token from the tokenizer
EOS_TOKEN = tokenizer.eos_token

#### Create a prompt template

In [ ]:
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

### Prompt Formatting

**What `prompt_formatter` does?**

* Takes a dataset row (Dialogues, Summary) and a prompt template (`prompt template`).
* Adds an instruction: `"Write a concise summary of the following dialogue."`
* Fills the template with **instruction + dialogue + summary**.
* Appends the **EOS token** to mark the end.
* Returns the final prompt as `{'text': formatted_prompt}` for training.

This ensures each example is structured like:
**Instruction - Dialogue - Summary \[EOS]**

In [ ]:
def prompt_formatter(example, prompt_template):
    # Instruction for the model
    instruction = 'Write a concise summary of the following dialogue.'

    # Extract dialogue and reference summary from the dataset example
    dialogue = example["Dialogues"]
    summary = example["Summary"]

    # Merge the instruction, dialogue, and summary into the prompt template
    # Append EOS_TOKEN to mark the end of the sequence
    formatted_prompt = prompt_template.format(instruction, dialogue, summary) + EOS_TOKEN

    # Return as a dictionary in the format expected by the trainer
    return {'text': formatted_prompt}


Notice how we are adding the end-of-sequence token to the prompt i.e. we're adding a special marker at the end of the prompt to show it's finished

In [ ]:
# Apply the prompt_formatter function to each example in the training dataset
# This formats dialogues and summaries into prompts suitable for model training
formatted_training_dataset = training_dataset.map(
    prompt_formatter,
    fn_kwargs={'prompt_template': alpaca_prompt}  # Pass the Alpaca-style prompt template
)


Map:   0%|          | 0/50 [00:00<?, ? examples/s]

In [ ]:
# Read the fine-tuning validation CSV into a Pandas DataFrame
validation = pd.read_csv("/content/finetuning_validation.csv")

# Convert the DataFrame into a dictionary of lists (required for Hugging Face Dataset)
validation_dict = validation.to_dict(orient='list')

# Create a Hugging Face Dataset from the dictionary
validation_dataset = Dataset.from_dict(validation_dict)

In [ ]:
# Apply the prompt_formatter function to each example in the validation dataset
# This formats dialogues and summaries into prompts suitable for model evaluation
formatted_validation_dataset = validation_dataset.map(
    prompt_formatter,
    fn_kwargs={'prompt_template': alpaca_prompt}  # Pass the Alpaca-style prompt template
)


Map:   0%|          | 0/10 [00:00<?, ? examples/s]

## Fine-Tuning

We now patch in the adapter modules to the base model using the `get_peft_model` method.


We are adapting the large language model for our task using a technique called **LoRA (Low-Rank Adaptation)**. Instead of retraining the entire model (which would be very expensive), LoRA only updates a small number of parameters while keeping most of the model frozen.


* **`r`** - Rank of low-rank matrices; higher = more adaptation, typical 4-64.
* **`lora_alpha`** - Scaling factor for LoRA updates; higher = stronger effect, typical 8-32.
* **`lora_dropout`** - Dropout on LoRA layers to prevent overfitting, 0-0.3.
* **`target_modules`** - The specific parts of the model we allow to be updated.
* **`use_gradient_checkpointing`** - Save memory by recomputing activations, `True`/`False`.
* **`random_state`** - Seed for reproducibility, any integer.

This step makes the model **lighter, faster, and cheaper to fine-tune**, while still learning how to summarize dialogues effectively.

For more information, please refer to the [Unsloth](https://github.com/unslothai/unsloth) repository.

**NOTE:** This is a LoRA model because we are only applying low-rank adapters on top of the frozen model weights. Although the base model is loaded in 4-bit precision, we are not using QLoRA’s specific quantization (NF4 + double quantization) or gradient handling required for QLoRA fine-tuning.

In [ ]:
# Convert the base model into a LoRA (adapter) fine-tunable model
model = FastLanguageModel.get_peft_model(
    model,
    r=16,                         # Rank of the LoRA update matrices
    lora_alpha=16,                # Scaling factor for LoRA updates   #  `r=16` and `lora_alpha=16` provide a balanced trade-off, giving enough capacity
                                                                      #    to learn task-specific patterns while keeping updates stable and efficient.
    lora_dropout=0,               # Dropout rate for LoRA layers (set 0 for fast patching support)
    bias="none",                  # How biases are handled (none = leave them unchanged)
    target_modules=[              # Model layers where LoRA adapters will be applied
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj"
    ],
    use_gradient_checkpointing=True,  # Saves memory during training by recomputing activations
    random_state=42                  # For reproducibility
)


Unsloth: Already have LoRA adapters! We shall skip this step.


The **architecture** of the Mistral model, specifically the MistralForCausalLM, consists of several key components:

1) Embedding Layer: The model starts with an embedding layer that converts input tokens into a dense representation with an output size of 4096, supporting a vocabulary of 32,000 tokens.

2) Decoder Layers: The core of the model comprises 32 MistralDecoderLayer instances, each containing:
- Self-Attention Mechanism: This includes multiple projection layers for queries,
keys, values, and output, all designed to handle 4-bit precision for efficient computation. Rotary embeddings are also employed for position encoding.
- Feedforward Network (MLP): The MLP features gates and projections to expand the dimensionality to 14,336 before reducing it back to 4096, using the SiLU activation function.
- Layer Normalization: Each decoder layer includes input and post-attention normalization using MistralRMSNorm.

3) Final Normalization: The entire model concludes with an additional normalization layer.

4) Linear Output Head: The model includes a linear layer that maps the 4096-dimensional output back to the token vocabulary size (32,000), enabling the generation of predictions.

In [ ]:
model

Notice how LoRA adapters are attached to the layers specified during instantiation.

```
PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): MistralForCausalLM(
      (model): MistralModel(
        (embed_tokens): Embedding(32000, 4096, padding_idx=0)
        (layers): ModuleList(
          (0-31): 32 x MistralDecoderLayer(
            (self_attn): MistralAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
                zzz(lora_dropout): ModuleDict(
                  (default): Identity()
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4096, out_features=1024, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Identity()
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=1024, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (v_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4096, out_features=1024, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Identity()
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=1024, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (o_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Identity()
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (rotary_emb): LlamaRotaryEmbedding()
            )
            (mlp): MistralMLP(
              (gate_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4096, out_features=14336, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Identity()
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=14336, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (up_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4096, out_features=14336, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Identity()
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=14336, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (down_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=14336, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Identity()
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=14336, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (act_fn): SiLU()
            )
            (input_layernorm): MistralRMSNorm((4096,), eps=1e-05)
            (post_attention_layernorm): MistralRMSNorm((4096,), eps=1e-05)
          )
        )
        (norm): MistralRMSNorm((4096,), eps=1e-05)
        (rotary_emb): LlamaRotaryEmbedding()
      )
      (lm_head): Linear(in_features=4096, out_features=32000, bias=False)
    )
  )
)
```



For training, we use the following nuances borrowed from the broader deep learning discipline.

- Low learning rates for smooth parameter updates
- Early stopping to monitor for validation loss (negative log likelihood in this case)
- Checkpointing to enable resumption of training


We are creating a **trainer** that will handle the fine-tuning of our model. The trainer takes care of feeding the data into the model, running the training loop, tracking progress, and saving results.

Key points in this setup:

* **Model & Tokenizer** - The language model and its tokenizer we are fine-tuning.
* **Training & Validation Data** - Split datasets so the model can learn on one set and be tested on another.
* **Max Sequence Length (2048)** - How much text the model can read at once.
* **Data Collator** - Groups the data into batches in the right format.
* **Batch Size & Gradient Accumulation** - Train on small pieces at a time (due to memory limits) and combine updates to act like a larger batch.
* **Learning Rate & Optimizer** - Control how fast the model learns and how updates are applied.
* **Epochs / Steps** - How long the model trains.
* **FP16 / BF16** - Use lower precision for faster and more memory-efficient training.
* **Output Directory** - Where trained model checkpoints and logs are saved.


This trainer automates the whole training process from sending data into the model to adjusting weights, logging progress, and saving results, making fine-tuning efficient and manageable.


In [ ]:
trainer = SFTTrainer(
    model = model,  # LoRA-adapted model to fine-tune
    tokenizer = tokenizer,  # Tokenizer corresponding to the model
    train_dataset = formatted_training_dataset,  # Training dataset in prompt-ready format
    eval_dataset = formatted_validation_dataset,  # Validation dataset for evaluation
    dataset_text_field = "text",  # Field in dataset containing the input text
    max_seq_length = 2048,  # Maximum sequence length for training
    data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer),  # Handles batching
    dataset_num_proc = 2,  # Number of processes for dataset preprocessing
    packing = False,  # Packing short sequences can make training faster (disabled here)
    args = TrainingArguments(
        per_device_train_batch_size = 2,  # Batch size per GPU/CPU
        gradient_accumulation_steps = 4,  # Accumulate gradients over steps to simulate larger batch
        warmup_steps = 5,  # Learning rate warmup steps
        max_steps = 30,  # Total training steps (used here for quick demonstration)
        learning_rate = 2e-4,  # Learning rate for optimizer
        fp16 = not is_bfloat16_supported(),  # Use 16-bit float if bfloat16 not supported
        bf16 = is_bfloat16_supported(),  # Use bfloat16 if supported
        logging_steps = 1,  # Log metrics every step
        optim = "adamw_8bit",  # 8-bit AdamW optimizer for memory efficiency
        weight_decay = 0.01,  # Regularization to prevent overfitting
        lr_scheduler_type = "linear",  # Linear learning rate decay
        seed = 3407,  # For reproducibility
        output_dir = "outputs",  # Directory to save checkpoints and outputs
        report_to = "none"  # No external logging (like WandB)
    ),
)


In [ ]:
training_history = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 50 | Num Epochs = 5 | Total steps = 30
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 41,943,040 of 7,283,675,136 (0.58% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,2.388400
2,2.406600
3,2.202300
4,1.798700
5,1.481300
6,1.229600
7,1.176100
8,0.687000
9,0.298700
10,0.145400


## Saving the Trained Model


We will be saving the **LoRA Parameters** of our fine-tuned model so that we can test/evaluate the model later. Since fine-tuning is an expensive process, it’s best to save these adapter files in case of crashes.


### Setup to enable bash commands

This code ensures that all file names and metadata are encoded in UTF-8, preventing errors when writing model files to disk or Google Drive.

In [ ]:
# Setup to ensure Python uses UTF-8 encoding for shell/batch commands
import locale

# Override the system's preferred encoding to always return "UTF-8"
def getpreferredencoding():
    return "UTF-8"

locale.getpreferredencoding = getpreferredencoding

In [ ]:
lora_model_name = "finetuned_mistral_llm"

In [ ]:
model.save_pretrained(lora_model_name)

`ls -lh {folder}`

* **ls** - Lists files and folders.
* **-l** - Shows detailed information like permissions, owner, size, and modification date.
* **-h** - Makes file sizes human-readable (KB, MB, GB instead of bytes).
* `{folder}` - The folder whose contents you want to see.

Shows the **contents and sizes** of a folder in a readable format.

In [ ]:
!ls -lh {lora_model_name}

total 161M
-rw-r--r-- 1 root root  950 Aug 20 18:11 adapter_config.json
-rw-r--r-- 1 root root 161M Aug 20 18:11 adapter_model.safetensors
-rw-r--r-- 1 root root 5.2K Aug 20 18:11 README.md


`cp -r {source} {destination}`

* **cp** - Stands for “copy”.
* **-r** - Means “recursive”, which allows copying **folders and all their contents** (subfolders and files).
* `{source}` - The folder you want to copy.
* `{destination}` - Where you want to copy it to.

Copies a folder and everything inside it to another location.




In [ ]:
# # Comment out this cell if you want to save the model to Google Drive

# from google.colab import drive
# drive.mount('/content/drive')

# drive_model_path = "/content/drive/MyDrive/finetuned_mistral_llm"

# !cp -r {lora_model_name} {drive_model_path}

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# **3. Evaluation of LLM after FineTuning**

### Loading the Fine-tuned Mistral LLM

In [ ]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=lora_model_name,                 # Replace the model name with "drive_model_path" if you are loading the model from Drive
    max_seq_length=2048,
    dtype=None,
    load_in_4bit=True
)

==((====))==  Unsloth 2025.8.9: Fast Mistral patching. Transformers: 4.51.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.4.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.32.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


### Inferencing

In [ ]:
alpaca_prompt_template = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Write a concise summary of the following dialogue.

### Input:
{}

### Response:
{}
"""

In [ ]:
predicted_summaries = []

In [ ]:
# Loop through each dialogue in the test set and generate summaries
for dialogue in tqdm(test_dialogues):
    try:
        # Format the dialogue into the Alpaca-style prompt
        prompt = alpaca_prompt_template.format(dialogue, '')

        # Tokenize the prompt and move to GPU
        inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

        # Generate model output (summary)
        outputs = model.generate(
            **inputs,
            max_new_tokens=128,          # Limit summary length
            use_cache=True,              # Reuse past key values for efficiency
            temperature=0,               # Deterministic output
            pad_token_id=tokenizer.eos_token_id
        )

        # Decode the generated tokens into text, skipping special tokens
        prediction = tokenizer.decode(
            outputs[0][inputs.input_ids.shape[-1]:],  # Remove input prompt tokens
            skip_special_tokens=True,
            cleanup_tokenization_spaces=True
        )

        # Store the generated summary
        predicted_summaries.append(prediction)

    except Exception as e:
        print(e)  # Log error if generation fails and continue
        continue

100%|██████████| 10/10 [00:32<00:00,  3.24s/it]


### Evaluation

In [ ]:
predicted_summaries

['Client inquired about coverage across multiple states and its impact on premiums and compliance. Action: Share case file and compliance checklist within 48 hours.',
 'Client inquired about cyber insurance options that protect against data breaches and operational downtime. Action: Share sample policy and incident response playbook.',
 'Client inquired about sustainability-linked incentives and its integration with employee benefit plans. Action: Share case study and usage analytics from a similar pilot program.',
 'Client inquired about long-term care insurance as a benefit for aging workforce and its integration as a voluntary benefit. Action: Send sample plan designs and communications kit.',
 'Client inquired about financial wellness support and its customizability. Action: Share case study, white-labeling options, and usage analytics.',
 'Client inquired about global travel insurance and its restrictions. Action: Share pricing matrix and activation guide.',
 'Client inquired abou

In [ ]:
# Evaluate the quality of generated summaries using BERTScore
score = bert_scorer.compute(
    predictions=predicted_summaries,  # Summaries generated by the model
    references=test_summaries,        # Ground-truth summaries from the dataset
    lang='en',                        # Specify English language
    rescale_with_baseline=True        # Normalize scores for easier interpretation
)


In [ ]:
# Compute the average F1 score across all test examples
avg_f1 = sum(score['f1']) / len(score['f1'])
avg_f1

0.5259339898824692

**The BERT Score of Finetuned Mistral LLM is 0.53**


# **Conclusion**

**We observed a significant improvement in the BERTScore after fine-tuning the Mistral model, also an observation can be made on the Predicted Summaries**

- Previously, the generated summaries of client interactions were overly verbose and lacked alignment with user preferences and domain-specific needs.
- By fine-tuning a language model on task-relevant and insurance-specific communication data, we significantly improved the model's ability to generate concise, actionable, and context-aware summaries.
- The fine-tuned model now produces outputs that are not only more relevant and structured but also tailored to user expectations, enhancing sales productivity and ensuring better client engagement in the insurance domain.

<font size = 6 color="#4682B4"><b> Power Ahead </font>
___